# Settings

In [1]:
%env TF_KERAS = 1
import os
sep_local = os.path.sep

import sys
sys.path.append('..'+sep_local+'..')
print(sep_local)

env: TF_KERAS=1
\


In [2]:
import tensorflow as tf
print(tf.__version__)

2.1.0


# Dataset loading

In [3]:
dataset_name='pokemon'

In [4]:
IMG_DIR = 'C:\\Users\\Khalid\\Documents\projects\\pokemon\DS06\\'
VAL_PCT = 20
VAL_FORMAT = 'png'

In [5]:
from training.generators.file_image_generator import create_image_lists, get_generators

Using TensorFlow backend.


In [6]:
imgs_list = create_image_lists(
    image_dir=IMG_DIR, 
    validation_pct=VAL_PCT, 
    valid_imgae_formats=VAL_FORMAT
)

  DEBUG    | Looking for images in 'all'


  INFO     | 809 file found


In [7]:
inputs_shape= IMG_SIZE=(200, 200, 3)
BATCH_SIZE = 32
latent_dim = 32
intermediate_dim = 32

In [8]:
train_gen, test_gen = get_generators(
    images_list=imgs_list, 
    image_dir=IMG_DIR, 
    image_size=IMG_SIZE, 
    batch_size=BATCH_SIZE, 
    class_mode=None
)

  INFO     | Found 738 training files


  INFO     | Found 71 validation files


In [9]:
import tensorflow as tf

In [10]:
train_ds = tf.data.Dataset.from_generator(
    lambda: train_gen, 
    output_types=tf.float32 ,
    output_shapes=tf.TensorShape((BATCH_SIZE, ) + IMG_SIZE)
)

test_ds = tf.data.Dataset.from_generator(
    lambda: test_gen, 
    output_types=tf.float32 ,
    output_shapes=tf.TensorShape((BATCH_SIZE, ) + IMG_SIZE)
)

In [11]:
_instance_scale=1.0
for data in train_ds:
    _instance_scale = float(data[0].numpy().max())
    break

In [12]:
_instance_scale

1.0

In [13]:
import numpy as np
from collections.abc import Iterable

In [14]:
if isinstance(inputs_shape, Iterable):
    _outputs_shape = np.prod(inputs_shape)

In [15]:
_outputs_shape

120000

# Model's Layers definition

In [16]:
menc_lays = [tf.keras.layers.Dense(units=intermediate_dim//2, activation='relu'),
            tf.keras.layers.Dense(units=intermediate_dim//2, activation='relu'),
            tf.keras.layers.Flatten(),
            tf.keras.layers.Dense(units=latent_dim)]

venc_lays = [tf.keras.layers.Dense(units=intermediate_dim//2, activation='relu'),
            tf.keras.layers.Dense(units=intermediate_dim//2, activation='relu'),
            tf.keras.layers.Flatten(),
            tf.keras.layers.Dense(units=latent_dim)]

dec_lays = [tf.keras.layers.Dense(units=latent_dim, activation='relu'),
            tf.keras.layers.Dense(units=intermediate_dim, activation='relu'),
            tf.keras.layers.Dense(units=_outputs_shape),
            tf.keras.layers.Reshape(inputs_shape)]

# Model definition

In [17]:
model_name = dataset_name+'VAEssmi'
recording_dir='..'+sep_local+'..'+sep_local+'recording'+sep_local+model_name

In [18]:
from training.autoencoding_basic.autoencoders.VAE import VAE as AE

In [19]:
inputs_shape=IMG_SIZE

In [20]:
variables_params = \
[
    {
        'name': 'inference_mean', 
        'inputs_shape':inputs_shape,
        'outputs_shape':latent_dim,
        'layers': menc_lays#enc_lays2#
    },
    
    {
        'name': 'inference_logvariance', 
        'inputs_shape':inputs_shape,
        'outputs_shape':latent_dim,
        'layers': venc_lays#enc_lays2#
    },
        {
        'name': 'generative', 
        'inputs_shape':latent_dim,
        'outputs_shape':inputs_shape,
        'layers':dec_lays#dec_lays2#
    }
]

In [21]:
from utils.data_and_files.file_utils import create_if_not_exist

In [22]:
_restore = os.path.join(recording_dir, 'var_save_dir')

In [23]:
create_if_not_exist(_restore)
_restore

'..\\..\\recording\\pokemonVAEssmi\\var_save_dir'

In [24]:
#to restore trained model, set filepath=_restore

In [25]:
ae = AE( 
    name=model_name,
    inputs_shape=inputs_shape,
    outputs_shape=inputs_shape,
    latent_dim=latent_dim,
    batch_size=BATCH_SIZE,
    variables_params=variables_params, 
    filepath=None
    )

Model: "inference_mean"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
inference_mean_inputs (InputLa [(None, 200, 200, 3)]     0         
_________________________________________________________________
dense (Dense)                (None, 200, 200, 16)      64        
_________________________________________________________________
dense_1 (Dense)              (None, 200, 200, 16)      272       
_________________________________________________________________
flatten (Flatten)            (None, 640000)            0         
_________________________________________________________________
dense_2 (Dense)              (None, 32)                20480032  
_________________________________________________________________
batch_normalization (BatchNo (None, 32)                128       
_________________________________________________________________
dropout (Dropout)            (None, 32)           

  WARNING  | None


Model: "inference_logvariance"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
inference_logvariance_inputs (Input [(None, 200, 200, 3)]     0         
_________________________________________________________________
dense_3 (Dense)              (None, 200, 200, 16)      64        
_________________________________________________________________
dense_4 (Dense)              (None, 200, 200, 16)      272       
_________________________________________________________________
flatten_1 (Flatten)          (None, 640000)            0         
_________________________________________________________________
dense_5 (Dense)              (None, 32)                20480032  
_________________________________________________________________
batch_normalization_1 (Batch (None, 32)                128       
_________________________________________________________________
dropout_1 (Dropout)          (None, 32

  WARNING  | None


Model: "generative"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
generative_inputs (InputLaye [(None, 32)]              0         
_________________________________________________________________
dense_6 (Dense)              (None, 32)                1056      
_________________________________________________________________
dense_7 (Dense)              (None, 32)                1056      
_________________________________________________________________
dense_8 (Dense)              (None, 120000)            3960000   
_________________________________________________________________
reshape (Reshape)            (None, 200, 200, 3)       0         
_________________________________________________________________
batch_normalization_2 (Batch (None, 200, 200, 3)       12        
_________________________________________________________________
dropout_2 (Dropout)          (None, 200, 200, 3)       0

  WARNING  | None


In [26]:
#from evaluation.quantitive_metrics.psnr import psnr
from evaluation.quantitive_metrics.ssmi import ssmi
#from evaluation.quantitive_metrics.sharp_diff import sharp_diff


In [27]:
ae.compile(loss={'x_logits': ssmi})

Model: "pokemonVAEssmi"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
inference_logvariance_inputs (InputLay [(None, 200, 200, 3) 0                                            
__________________________________________________________________________________________________
inference_logvariance (Model)          (None, 32)           20480496    inference_logvariance_inputs[0][0]      
__________________________________________________________________________________________________
tf_op_layer_mul (TensorFlowOpLa [(None, 32)]         0           inference_logvariance[1][0]             
__________________________________________________________________________________________________
tf_op_layer_Exp (TensorFlowOpLa [(None, 32)]         0           tf_op_layer_mul[0][0]            
_________________________________________________________

# Callbacks

In [28]:
from training.callbacks.progress_bar import NotebookPrograssBar
from training.callbacks.sample_generation import SampleGeneration
from training.callbacks.save_model import ModelSaver

In [29]:
progbar = NotebookPrograssBar(leave_outer=False)

In [30]:
es = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss', 
    min_delta=1e-12, 
    patience=5, 
    verbose=1, 
    restore_best_weights=True
)

In [31]:
ms = ModelSaver(filepath=_restore,save_freq=5)

In [32]:
csv_dir = os.path.join(recording_dir, 'csv_dir')
create_if_not_exist(csv_dir)
csv_dir = os.path.join(csv_dir, ae.name+'.csv')
csv_log = tf.keras.callbacks.CSVLogger(csv_dir, append=True)
csv_dir

'..\\..\\recording\\pokemonVAEssmi\\csv_dir\\pokemonVAEssmi.csv'

In [33]:
image_gen_dir = os.path.join(recording_dir, 'image_gen_dir')
create_if_not_exist(image_gen_dir)

In [34]:
sg = SampleGeneration(latent_shape=latent_dim, filepath=image_gen_dir, gen_freq=5, save_img=True, gray_plot=False)

In [35]:
import numpy as np

# Model Training

In [ ]:
ae.fit(
    x=train_ds,
    input_kw=None,
    steps_per_epoch=750,
    epochs=int(1e6), 
    verbose=0,
    callbacks=[progbar, es, ms, csv_log, sg],
    workers=-1,
    use_multiprocessing=True,
    validation_data=test_ds,
    validation_steps=250
)